In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
# from pyspark.ml.classification import Mul
from pyspark.mllib.tree import RandomForest
from pyspark.ml.classification import RandomForestClassifier

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from time import gmtime, strftime
 
TIMESTAMP = strftime("%d%m%Yt%H%M%Sz", gmtime())
print(TIMESTAMP)
ID = "vkbomb" #my gfk ID    
    
spark = SparkSession.builder \
.master("local[*]") \
.appName("{}_app_{}".format(ID,TIMESTAMP)) \
.config("spark.driver.memory", "500g") \
.config("spark.executer.memory", "500g") \
.config("spark.submit.deployMode","client") \
.enableHiveSupport() \
.getOrCreate()

16062019t140304z


In [30]:
# # Prepare training documents from a list of (id, text, label) tuples.
# df_training = spark.createDataFrame([
#     (0, "a", 1.0),
#     (1, "b", 0.0),
#     (2, "a", 1.0),
#     (3, "b", 0.0)
# ], ["id", "text", "label"])

# # df_training = spark.createDataFrame([
# #     (0, "a b c d e spark", 1.0),
# #     (1, "b d", 0.0),
# #     (2, "spark f g h", 1.0),
# #     (3, "hadoop mapreduce", 0.0)
# # ], ["id", "text", "label"])

In [109]:

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
# lr = LogisticRegression(maxIter=10, regParam=0.001)
rf = RandomForestClassifier(labelCol='label',featuresCol='features', numTrees=1)



pipeline = Pipeline(stages=[tokenizer, hashingTF, rf])


In [110]:
# pipeline

In [111]:
# model = pipeline.fit(df_training)


In [112]:
# # Prepare test documents, which are unlabeled (id, text) tuples.
# df_test = spark.createDataFrame([
#     (4, "a"),
#     (5, "b"),
#     (6, "a"),
#     (7, "c")
# ], ["id", "text"])

In [50]:
# # Make predictions on test documents and print columns of interest.
# prediction = model.transform(df_test)
# prediction.show()
# # selected = prediction.select("id", "text", "probability", "prediction")
# # for row in selected.collect():
# #     rid, text, prob, prediction = row
# #     print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))


+---+----+-----+--------------------+-------------+-----------+----------+
| id|text|words|            features|rawPrediction|probability|prediction|
+---+----+-----+--------------------+-------------+-----------+----------+
|  4|   a|  [a]|(262144,[227410],...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|  5|   b|  [b]|(262144,[30913],[...|    [1.0,0.0]|  [1.0,0.0]|       0.0|
|  6|   a|  [a]|(262144,[227410],...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|  7|   c|  [c]|(262144,[28698],[...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
+---+----+-----+--------------------+-------------+-----------+----------+



In [3]:

from pyspark.sql.types import IntegerType

sdf = spark.read.csv("file:////home/vkbomb/working/data/train_32353_new_cleaned_item_id.csv",sep=",",header=True)

sdf_filtered = sdf.select("main_text","item_id")
sdf_filtered = sdf_filtered.withColumn("item_id",sdf_filtered["item_id"].cast(IntegerType())).fillna(0)
# sdf.printSchema()

In [4]:
sdf_test = spark.read.csv("file:////home/vkbomb/working/data/test_32353_new.csv",sep=",",header=True)
# sdf_test.printSchema()
sdf_test = sdf_test.select("main_text")

In [5]:
from pyspark.mllib.tree import RandomForest
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier


# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="main_text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
strIndexer = StringIndexer(inputCol='item_id',outputCol='indexer_item_id')

# lr = LogisticRegression(maxIter=10, regParam=0.001)
rf = RandomForestClassifier(labelCol='indexer_item_id',featuresCol='features', numTrees=1) #'item_id'

dt = DecisionTreeClassifier(labelCol="indexer_item_id", featuresCol="features")


In [6]:
pipeline = Pipeline(stages=[strIndexer, tokenizer, hashingTF,dt]) #, rf

In [7]:
(sdf_filtered_1, sdf_filtered_2) = sdf_filtered.randomSplit([0.2,0.8])
# (sdf_test_1, sdf_test_2) = sdf_test.randomSplit([0.5,0.5])

In [ ]:
model = pipeline.fit(sdf_filtered_1)
# prediction = model.transform(sdf_test)
# prediction.show()